In [14]:
#####
#Note
#####
#There is more functionality in the draft version - this is just a simple working version that's organised

#Uncomment to install the following packages
#%pip install googletrans==4.0.0-rc1 
#%pip install unidecode
#%pip install langdetect

#APIs and Libraries used for transaltion
from googletrans import Translator
from langdetect import detect
from unidecode import unidecode

#Imports
import os
import time

#Initialised translator
translator = Translator()
translator.raise_Exception = True

def runScript():
    path = "C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy"  #getDirPath()
    trasnlateFilesInDirectoy(path)

def trasnlateFilesInDirectoy(path):
    fullFilePaths, rootOfPaths, nameOfFiles = listFilePaths(path)
    trasnlatedFileNames = translateList(nameOfFiles, path) #The 2nd arguements specifies where to write the text files showing all the before and after translations
    trasnlatedFileNames = resolveDuplicates(trasnlatedFileNames)

    i = 0
    for fullP, rootP, tranlatedFN in zip(fullFilePaths, rootOfPaths, trasnlatedFileNames):
        os.rename(fullP, os.path.join(rootP, (tranlatedFN + ".mp3")).replace("\"", "'").replace("?", "？"))

        #To print progress
        print("[" + str(i) + "/" + str(len(fullFilePaths)) + "]: " + str(os.path.join(rootP, (tranlatedFN + ".mp3")).replace("\"", "'").replace("?", "？")[:250]))
        i += 1
    
    print("> Done")

#################
#HELPER FUNCTIONS
#################

#Gets and formats the path of the directory that contains the MP3 files (and sub directories with MP3 files) that we want to rename with translations
def getDirPath():
    #Asks the user for a path to do the operations on
    path = input("File Path: ")

    #If there is no backslash in the path, the bath is not in the windows user path and the path doesn't exist, a new path is requested
    while "\\" not in path and ":\\Users" not in path and os.path.isdir(path):
        print("File Path Invalid")
        path = input("Please Select A Different File Path: ")
    #Gets rid of trailing backslashes or any file names attached to the end of the path
    if path.split("\\")[-1] == "" or "." in path.split("\\")[-1]:
        path = path.rsplit("\\", 1)[0]

#Gets, stores and returns the a list of filenames, parent directories and full paths or all the files in the given path and it's sub directories 
def listFilePaths(path):
    #These hold the filenames and the directories they're contained within seperarely and together
    fullFilePaths = [] 
    rootOfPaths = [] 
    nameOfFiles = []

    #Recursively goes through all directories and finds files within them
    for root, directories, files in os.walk(path):
        for file in files:
            fullFilePaths.append(os.path.join(root, file)) #The full path for each file is stored here
            nameOfFiles.append(file.rsplit(".", 1)[0]) #The names of each file found is stored here - the file extension is removes so it is not transalted
            rootOfPaths.append(root) #The path to each file's parent directory is stored her - this is prepended to the transalted filename later
    
    return fullFilePaths, rootOfPaths, nameOfFiles

#Translates list of filesnames
def translateList(untranslatedList, logPath):
    trasnlatedFileNames = []

    #For primitive progress bar
    ###########################################
    #print("[", end="") 
    #noOfTrasnlationNeed = len(untranslatedList)
    #noOfTrasnlationsNeedForProgresTick = int(noOfTrasnlationNeed / 100)
    #progress = 0
    ###########################################

    translationNo = 1
    logBehaviour("##Start Of Logs##\n#################\n", logPath)

    #Trasnlates filenames to english
    for fileName in untranslatedList:
        print(logBehaviour("> ", logPath), end="")
        time.sleep(0.5) #Puts the script to sleep inbetween translations to avoid HTTP 429 errors from sending too many requests at once
        #If filename is already fully in english, it is not traslated
        if isEnglish(fileName):
            trasnlation = fileName
            trasnlatedFileNames.append(fileName)
        else:
            #A 429 error could occur if too many requests are made at once, so when that happens, the program is put to sleep (takes a short break) and then resumes
            try: 
                trasnlation = translator.translate(fileName[:250], dest="en").text[:250]
                trasnlatedFileNames.append(translator.translate(fileName[:250], dest="en").text[:250])

            except:
                noOfTicks = 60
                sleepLen = 5
                print(
                    "A HTTP 429 Error Occured. Too many translation requests were sent to the google API in a short period of time." +
                    "Pausing script and resuming in about " + str(int((noOfTicks * sleepLen)/60)) + " minutes to attempt to rectify issue. \n>" + 
                    "Note that if the script end with an error/exception again, the daily qouta of trasnlation the google API allows has likely been reached." + 
                    "Note that none of these translations have been used to rename anything yet.\n" +
                    "[", 
                end="")
                for i in range(noOfTicks):
                    time.sleep(sleepLen)
                    print("#", end="")
                print("]\n>> Attempting to Resume! <<\n> ", end="")

                trasnlation = translator.translate(fileName[:250], dest="en").text[:250]
                trasnlatedFileNames.append(translator.translate(fileName[:250], dest="en").text[:250])
                print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + fileName + "  \n\t\t===>  " + trasnlation, logPath))
                translationNo += 1
                continue
        
        print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + fileName + "  \n\t\t===>  " + (trasnlation if trasnlation != fileName else "[[Same as before - no translation]]"), logPath))
        translationNo += 1

        #For primitive progress bar
        ##########################################
        #progress += 1
        #if (progress % noOfTrasnlationsNeedForProgresTick) == 0:
        #    print("#", end="") 
        #if progress == noOfTrasnlationNeed:
        #    print("]")
        ##########################################
    return trasnlatedFileNames

#Checks for duplicates in the list of filename translations and resolves them by adding a suffix - which makes it unique
def resolveDuplicates(uncheckedList):
    duplicates = []
    for item in uncheckedList:
        if uncheckedList.count(item) > 1:
            #Makes note the duplicate
            duplicates.append(item)

    #Makes duplicates list distinct - removes duplicates from it
    duplicates = list(set(duplicates))

    #Handles duplicates if any are found
    if duplicates:
        #Goes through each duplicate name found and results all the duplicates with the name
        for dup in duplicates:
            #Character to within in the suffix "-[]" - which is appended to duplicates
            duplicateSuffixChar = "I"
            duplicateSuffix = "-[]"

            suffixRepeater = 1 #Keeps track of the amount of "I"s we want in the "-[]" suffix 

            #Gives each ducplidate item a suffix
            for i in range(len(uncheckedList)):
                if uncheckedList[i] == dup:
                    duplicateSuffix = duplicateSuffix[:2] + duplicateSuffixChar + duplicateSuffix[2:] #Adds 'I' within suffix before appending to the filename 
                    suffixRepeater += 1 #Insures the suffix will have an extra "I" when the next duplicate is found
                    uncheckedList[i] = uncheckedList[i] + duplicateSuffix #Adds suffix to translated file
                    
        #Checks if the duplicate handler introduced duplicates
        return resolveDuplicates(uncheckedList)
    else:
        return uncheckedList

#checks if a string is fully english
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def logBehaviour(textToLog, logPath):
    logPath = logPath.rsplit("\\", 1)[0] + "\\EngToAsiLangTransLog.txt"
    if os.path.exists(logPath) and textToLog == "##Start Of Logs##\n#################\n":
        os.remove(logPath) 

    with open(logPath, 'a+', encoding='utf8') as file: 
        file.write(textToLog)
        file.close()

    return textToLog


##########################
#Main function placeholder
##########################
runScript()

> [1/943]: DRAMA CD vol5  
		===>  [[Same as before - no translation]]
> [2/943]: SERVANT x SERVICE x WORKING!! ドラマCD  
		===>  SERVANT X SERVICE X Working !! Drama CD
> [3/943]: 【のうりん】 ドラマCD 【アニメ‘ｓ jp】  
		===>  【Uri】 Drama CD 【Anime 'S JP】
> [4/943]: 【ドラマＣＤ】【作業用】俺の彼女と幼なじみが修羅場すぎる 間島淳司 豊崎愛生 斉藤千和  
		===>  [Drama CD] [For work] My girlfriend and childhood friend are too shambles Mid Island Toyosaki Aisei Saito Saito
> [5/943]: 【ドラマＣＤ】【作業用】食戟のソーマ１１巻 同梱 松岡禎丞 種田利沙高橋未奈美  
		===>  【Drama CD】 【For work】 Soma Soma 11 volumes of food
> [6/943]: 【作業用BGM】中二病でも恋がしたいドラマCD集  
		===>  [Working BGM] Drama CD collection I want to fall in love with two illnesses
> [7/943]: 【熟肉】龙王的工作！第十一卷广播剧Drama CD  
		===>  [Cooked meat] Dragon King's work!Eleventh Volume Radio Drama Drama CD
> [8/943]: えむえむっ! ドラマCD「えむびよりっ!」  
		===>  Emue Mu! Drama CD "Esubi!"
> [9/943]: お宝音声？【百人一首】田中理恵の百人一首  
		===>  Treasure voice?[Hyakuni one neck] Hyakemono of Rie Tanaka
> [10/943]: はたらく魔王さま! ドラマCD『 魔王、捨て猫を拾う』  
		===>  The Demon 